# Assembling the TOP dataset

### Import libraries

In [ ]:
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt
import glob
import sys
sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep
from cvasl.file_handler import Config

## Read data into pandas dataframe

How do we define which files should be used together?
options:
- all files in folder
- based on suffix (e.g. "n=895_06-Feb-2023_PVC2.tsv")
- check first columns to see whether it matches

For now, we will will use the first option, approved by Mathijs on 26th June 2023

In [ ]:
experiment_folder= 'TOP'

In [ ]:
config = Config()
root_directory = config.get_directory('raw_data')
if os.path.isdir(os.path.join(root_directory, experiment_folder)):
    print("this folder exists, we will take tsv from here")
else: 
    print("this folder does not seem to exist, try typing again")

In [ ]:
root_directory = config.get_directory('raw_data')
tsv_path = os.path.join(root_directory, experiment_folder)

tsv_files = [os.path.join(tsv_path, file) for file in os.listdir(tsv_path) if file.endswith('.tsv')]
tsv_files

In [ ]:
#CoV files -> COV values NOT 
tsv_files_for_cov = []
basics = []
for file in os.listdir(tsv_path):
    if file.startswith('CoV') and file.endswith('.tsv'):
        tsv_files_for_cov.append(os.path.join(tsv_path, file))
    elif file.endswith('.tsv'):
        basics.append(os.path.join(tsv_path, file))
basics      

In [ ]:
# Read files in selected folder into dataframes
cov_dataframes = [pd.read_csv(file, sep='\t', header=[0]) for file in tsv_files_for_cov]
# make a sample
sample_cov_df = cov_dataframes[3] # example of COV file
cols_cov = sample_cov_df.columns
#look at sample
sample_cov_df

In [ ]:
# Read files in selected folder into dataframes
basic_dataframes = [pd.read_csv(file, sep='\t', header=[0]) for file in basics]
# make a sample
sample_basic_df = basic_dataframes[2] # example of COV file
cols_basic = sample_basic_df.columns
#look at sample
sample_basic_df

In [ ]:
# Read ALL files in selected folder into dataframes
dataframes = [pd.read_csv(file, sep='\t', header=[0]) for file in tsv_files]
# make a sample
sample_df = dataframes[2] # example of file
cols = sample_df.columns
#look at sample
#sample_df

## Below we can see some different columns should be in our final dataframe except sex and age, which we add from anotehr frame.
## These lists have some overalpe as some values are CoV values that are in are basic values

In [ ]:
#basic_dataframes

In [ ]:
list_all_basic_column = []
for frame in basic_dataframes:
    list_columns = frame.columns.to_list()
    for x in list_columns:
        if x not in list_all_basic_column:
            list_all_basic_column.append(x)
#list_all_basic_column

In [ ]:
list_all_cov_column = []
for frame in cov_dataframes:
    list_columns = frame.columns.to_list()
    for x in list_columns:
        if x not in list_all_cov_column:
            list_all_cov_column.append(x)
#list_all_cov_column

In [ ]:
#Add a function for COV columns to add _cov to name if units are SD/mean

In [ ]:
# Read files in selected folder into dataframes
cov_dataframes = [pd.read_csv(file, sep='\t', header=[0]) for file in tsv_files_for_cov]
# make a sample
sample_cov_df = cov_dataframes[2] # example of COV file
cols_cov = sample_cov_df.columns
#look at sample
sample_cov_df

# need to get all cov_columns?

In [ ]:
# we transform one of the COV, we need to transform all of them
for frame in cov_dataframes:
    print(frame.columns)

In [ ]:

dict = {
    'participant_id' : 'participant_id', #
    'session' : 'session', #,
    'LongitudinalTimePoint' : 'LongitudinalTimePoint', #
    'SubjectNList' : 'SubjectNList', #
    'Site' : 'Site', #
    'GM_vol' : 'GM_vol', #
    'WM_vol' : 'WM_vol' ,#
    'CSF_vol' : 'CSF_vol', #
    'GM_ICVRatio' : 'GM_ICVRatio', #
    'GMWM_ICVRatio' : 'GMWM_ICVRatio', #
    'WMH_vol' : 'WMH_vol',#
    'WMH_count' : 'WMH_count', # to here
    'ACA_proximal_B' : 'ACA_proximal_B_cov', #
    'ACA_proximal_L' : 'ACA_proximal_L_cov', #
    'ACA_proximal_R' : 'ACA_proximal_R_cov', #
    'ACA_intermediate_B' : 'ACA_intermediate_B_cov' ,#
    'ACA_intermediate_L' : 'ACA_intermediate_L_cov' ,#
    'ACA_intermediate_R' : 'ACA_intermediate_R_cov' ,#
    'ACA_distal_B' : 'ACA_distal_B_cov', #
    'ACA_distal_R' : 'ACA_distal_R_cov', #
    'MCA_proximal_B' : 'MCA_proximal_B_cov' ,#
    'MCA_proximal_L' : 'MCA_proximal_L_cov' ,#
    'MCA_proximal_R' : 'MCA_proximal_R_cov' ,#
    'MCA_intermediate_B' : 'MCA_intermediate_B_cov',#
    'MCA_intermediate_L' : 'MCA_intermediate_L_cov',#
    'MCA_intermediate_R' : 'MCA_intermediate_R_cov',#
    'MCA_distal_B' : 'MCA_distal_B_cov',#
    'MCA_distal_L' : 'MCA_distal_L_cov',#
    'MCA_distal_R' : 'MCA_distal_R_cov',#
    'PCA_proximal_B' : 'PCA_proximal_B_cov',#
    'PCA_proximal_L' : 'PCA_proximal_L_cov',#
    'PCA_proximal_R' : 'PCA_proximal_R_cov',#
    'PCA_intermediate_B' : 'PCA_intermediate_B_cov',#
    'PCA_intermediate_L' : 'PCA_intermediate_L_cov',
    'PCA_intermediate_R' : 'PCA_intermediate_R_cov',
    'PCA_distal_B' : 'PCA_distal_B_cov',
    'PCA_distal_L' : 'PCA_distal_L_cov',
    'PCA_distal_R' : 'PCA_distal_R_cov',
       }
sample_cov_df.rename(columns=dict,
          inplace=True)
sample_cov_df

### Below we take all the dataframes and put them into tsv files in a specific folder

In [ ]:
# numbr = 0
# for frame in dataframes:
#     filepath = '../open_work/internal_results/stitchy' 
#     filename = os.path.join(filepath,str(numbr+1)) 
#     if not os.path.exists(filepath):
#     # if filder doesn't exist, create it
#         os.makedirs(filepath)
#     frame.to_csv((filename +'.tsv'), sep="\t")
#     numbr +=1

In [ ]:
numbr = 0
for frame in cov_dataframes:
    filepath = '../open_work/internal_results/stitchy/cov' 
    filename = os.path.join(filepath,str(numbr+1)) 
    if not os.path.exists(filepath):
    # if filder doesn't exist, create it
        os.makedirs(filepath)
    frame.to_csv((filename +'.tsv'), sep="\t")
    numbr +=1

In [ ]:
numbr = 0
for frame in basic_dataframes:
    filepath = '../open_work/internal_results/stitchy/basics' 
    filename = os.path.join(filepath,str(numbr+1)) 
    if not os.path.exists(filepath):
    # if filder doesn't exist, create it
        os.makedirs(filepath)
    frame.to_csv((filename +'.tsv'), sep="\t")
    numbr +=1

In [ ]:
cov_tsv_folder_made = '../open_work/internal_results/stitchy/cov'
cov_identical_columns = sep.check_identical_columns(cov_tsv_folder_made)
cov_identical_columns

In [ ]:
basics_tsv_folder_made = '../open_work/internal_results/stitchy/basics'
basics_identical_columns = sep.check_identical_columns(basics_tsv_folder_made)
basics_identical_columns

In [ ]:
# now we need to make a super dataframes list
super_dataframes = basic_dataframes +cov_dataframes
len(super_dataframes)

## Copy identical columns from any file

we could also read it from the data, but if it's always the same, we can just define it here

In [ ]:
# tsv_folder_made = '../open_work/internal_results/stitchy'
identical_columns = sep.check_identical_columns(basics_tsv_folder_made)

In [ ]:
#identical_columns

In [ ]:
#sample_df.columns

In [ ]:
stitched = sample_basic_df[identical_columns].copy()

In [ ]:
n_identical = stitched.shape[1]

In [ ]:
stitched

### Here is where we add the different parts to stitched

In [ ]:
for df in super_dataframes:
    for col in df.columns[n_identical:]:
        stitched[col] = df[col]

stitched.columns

In [ ]:
len(stitched.columns)

## Here we can should get rid of second visits, but what we see is that session was not in the common columns. We will get rid of all second time points, and people ending in _2. And mention to scientists

In [ ]:
#stitched.columns

In [ ]:
# now we can get rid of double header
stitched = stitched[1:]
#stitched

In [ ]:
stitched['session'].unique()

In [ ]:
stitched['LongitudinalTimePoint'].unique()

### so due the fact they are all on first visit, first session, we can write this into p[atient id LATER

## So in this dataset we have one longitudinal timepoint, and one type of session. no need to filter down away from this

In [ ]:
stitched.columns

In [ ]:
sexage_path = os.path.join(root_directory, 'age_data', 'Age_sex_TOP.csv')
sexage_df = pd.read_csv(sexage_path, index_col=0)
sexage_df['renumber'] = sexage_df.index
sexage_df['renumber'] = sexage_df['renumber'].apply(str)
sexage_df

## Here we take the patient ID and align it with our other frame's index

In [ ]:
stitched = stitched.reset_index(drop=False)
stitched

In [ ]:
len(stitched.participant_id.unique())

In [ ]:
stitched['renumber'] = stitched['participant_id']

In [ ]:
sexage_df = sexage_df.reset_index(drop=True)
sexage_df

In [ ]:
#stitched.columns

In [ ]:
result = stitched.merge(sexage_df, on="renumber")
result

In [ ]:
result

In [ ]:
# just in case there are duplicates in there
result = result.loc[:,~result.columns.duplicated()].copy()

In [ ]:
result

## Conform file to new standard

### example of new standard (from M.D. on 23/08/2023)

In [ ]:
standard_path = '../researcher_interface/sample_sep_values/showable_standard.csv'


In [ ]:
standard = pd.read_csv(standard_path)
set_standard = set(standard.columns.to_list())

In [ ]:
result.columns = result.columns.str.lower()
set_results= set(result.columns.to_list())

z = set_results.intersection(set_standard) 
#z

In [ ]:
# # show values that are not as in template, may need recoding
# set_results - z

In [ ]:
#standard.columns

In [ ]:
#result.columns

In [ ]:
# shift column 'Name' to first position
first_column = result.pop('participant_id')
  
# insert column using insert(position,column_name,
# first_column) function
result.insert(0, 'participant_id', first_column)
result['participant_id'] = result['participant_id']+'_ses-1_run-1'
#result

In [ ]:
#standard.columns

In [ ]:
result['session_id'] = result['session']
second_column = result.pop('session_id')
result.insert(1, 'session_id', second_column)
result['site'] = "TOP"
result['run_id'] = result['longitudinaltimepoint']
#result

In [ ]:
#standard.columns

In [ ]:
result['gm_icv_ratio'] = result['gm_icvratio']
result['gmwm_icv_ratio'] = result['gm_icvratio']
result['cbf_gm_pvc0'] = 'NaN' 
result['cbf_gm_pvc2']=  result['totalgm_b'] 
result['cbf_wm_pvc0']= 'NaN'
result['cbf_wm_pvc2']= result['deepwm_b']
result['cbf_aca_pvc0'] =  'NaN'
result['cbf_mca_pvc0']  = 'NaN'
result['cbf_pca_pvc0'] =  'NaN'
result['cbf_aca_pvc2']  = result['aca_b']
result['cbf_mca_pvc2']  = result['mca_b']
result['cbf_pca_pvc2']  = result['pca_b']

In [ ]:
# for now we use proximals- need to check with student and scientists!
result['cov_gm_pvc0'] =  'NaN' # correct does not exist
# result['cov_gm_pvc2']  = result[]
result['cov_wm_pvc0']  = 'NaN' # does not exist
# result['cov_wm_pvc2']  = result[]
result['cov_aca_pvc0'] =  'NaN'# does not exist
result['cov_mca_pvc0']  = 'NaN'# does not exist
result['cov_pca_pvc0']  = 'NaN'# does not exist
result['cov_aca_pvc2'] = result['ACA_proximal_B_cov']
result['cov_mca_pvc2'] = result['MCA_proximal_B_cov']
result['cov_pca_pvc2'] = result['PCA_proximal_B_cov']


     # 'ACA_proximal_B_cov',
     #   'ACA_proximal_L_cov', 'ACA_proximal_R_cov', 'ACA_intermediate_B_cov',
     #   'ACA_intermediate_L_cov', 'ACA_intermediate_R_cov', 'ACA_distal_B_cov',
     #   'ACA_distal_R_cov', 'MCA_proximal_B_cov', 'MCA_proximal_L_cov',
     #   'MCA_proximal_R_cov', 'MCA_intermediate_B_cov',
     #   'MCA_intermediate_L_cov', 'MCA_intermediate_R_cov', 'MCA_distal_B_cov',
     #   'MCA_distal_L_cov', 'MCA_distal_R_cov', 'PCA_proximal_B_cov',
     #   'PCA_proximal_L_cov', 'PCA_proximal_R_cov', 'PCA_intermediate_B_cov',
     #   'PCA_intermediate_L_cov', 'PCA_intermediate_R_cov', 'PCA_distal_B_cov',
     #   'PCA_distal_L_cov', 'PCA_distal_R_cov'

In [ ]:
result.columns


In [ ]:
# we added a bunch of columns 
len(result.columns)

In [ ]:
# set_results= set(result.columns.to_list())

# z = set_results.intersection(set_standard) 
# new_result_columns = []
# for listable in z:
#     new_result_columns.append(listable)
# new_results = result[new_result_columns]
# new_results.columns

In [ ]:
# standard.columns[:33]

In [ ]:
# new_results = new_results[standard.columns[:33]]
# new_results

In [ ]:
sex_mapping = {0:'F',1:'M',}
results = result.assign(sex = result.sex.map(sex_mapping))
results

## here need to reorder the columns again

In [ ]:
standard.columns

In [ ]:
#results.columns

In [ ]:
len(results.columns)

In [ ]:
# shift columns to  position
third_column = results.pop('run_id')
fourth_column = results.pop('age')
fifth_column = results.pop('sex')
sixth_column = results.pop('site')
seventh_column = results.pop('gm_vol')
eight_column = results.pop('wm_vol')
ninth_column = results.pop('csf_vol')
tenth_column = results.pop('gm_icv_ratio')
eleventh_column = results.pop('gmwm_icv_ratio')
twelvth_column = results.pop('wmh_vol')
thirteenth_column = results.pop('wmh_count')

#last_column = results.pop('index')

results.insert(2, 'run_id', third_column)
results.insert(3, 'age', fourth_column)
results.insert(4, 'sex', fifth_column)
results.insert(5, 'site', sixth_column)
results.insert(6, 'gm_vol', seventh_column)
results.insert(7, 'wm_vol', eight_column)
results.insert(8, 'csf_vol', ninth_column)
results.insert(9, 'gm_ivc_ratio',tenth_column)
results.insert(10, 'gmwm_ivc_ratio',eleventh_column)
results.insert(11, 'wmh_vol',twelvth_column)
results.insert(12, 'wmh_count',thirteenth_column)
#results.insert(82, 'index', last_column)

results

In [ ]:
results = results.drop(['index', 'subjectnlist','session','longitudinaltimepoint', 'tp','gmwm_icvratio','gm_icvratio'], axis=1) 

In [ ]:
# for column in results.columns:
#     print(column, type(results[column][9]))
standard.columns

In [ ]:
results.columns

## now take last 20 columns, and put them after wmh count
not done yet

In [ ]:
pd.__version__

In [ ]:
head_columns = results.columns[:12]
middle_columns = results.columns[12:56]
tail_columns = results.columns[56:]

In [ ]:
f_results = pd.concat([results[head_columns],results[tail_columns],results[middle_columns]], axis=1)
f_results.columns

In [ ]:
# for column in results.columns:
#     print(column)

In [ ]:
for column in f_results.columns[6:]:
    #print(column)
    f_results[column] = pd.to_numeric(f_results[column], errors = 'coerce')


In [ ]:
f_results

In [ ]:


def find_outliers_by_list(dataframe, column_list):
    """
    This function finds the outliers in terms of anything outside two
    standard deviations
    from the mean on a list of specific specific column,
    then returns these rows of the dataframe.
    :param dataframe: whole dataframe on dataset
    :type dataframe: ~pandas.DataFrame
    :param column_list: list of relevant columns
    :type column_list: list
    :returns: dataframe of outliers
    :rtype: ~pandas.DataFrame
    """
    outlier_frames = []
    for column_n in column_list:
        mean = dataframe[column_n].mean()
        std = dataframe[column_n].std()
        values = dataframe[column_n].abs() - abs(mean + 2 * std)
        outliers = dataframe[values > 0]
        #outliers.style.applymap(highlight_cols, subset=pd.IndexSlice[:, [column_n]])
        #outliers.style.set_properties(subset=[column_n], **{'background-color': 'green'})
        outlier_frames.append(outliers)
        
    outlier_super = pd.concat(outlier_frames)
    return outlier_super


def check_sex_dimorph_expectations(dataframe):
    """
    This function checks that men
    as expected have larger brains than women
    in a given dataframe.
    """
    ladies = dataframe[dataframe['sex'] == 'F']
    men = dataframe[dataframe['sex'] == 'M']
    print('You have', len(ladies)/len(men), 'times as many ladies than men')
    if ladies.gm_vol.mean() < men.gm_vol.mean():
        print('As expected men have larger grey matter')
    if ladies.wm_vol.mean() < men.wm_vol.mean():
        print('As expected men have larger white matter')
    if ladies.gm_vol.mean() >= men.gm_vol.mean():
        print(
            'Caution, average female grey matter may be \
                  at similar or larger size than men'
        )
    if ladies.wm_vol.mean() >= men.wm_vol.mean():
        print(
            'Caution, average female white matter may be \
                  at similar or larger size than men'
        )
    if ladies.gm_vol.mean() >= men.gm_vol.mean() \
            or ladies.wm_vol.mean() >= men.wm_vol.mean():
        bad_data = dataframe
    else:
        bad_data = 0
    return bad_data

In [ ]:
sep.check_sex_dimorph_expectations(f_results)

In [ ]:
len(f_results.columns.to_list())

In [ ]:
#numeric_results.columns

In [ ]:
numeric_results = results.iloc[:, 2:55]
numeric_results = numeric_results.drop('sex', axis= 1) 
numeric_results = numeric_results.drop('site', axis= 1) 
#numeric_results = numeric_results.drop('renumber', axis= 1) 
#numeric_results = numeric_results.dropna(axis=0) 

In [ ]:
# maybe examine graphs?
sep.relate_columns_graphs(numeric_results, 'age')

In [ ]:
sep.relate_columns_graphs_numeric(f_results, 'age')

In [ ]:
doubled_outliers =sep.find_outliers_by_list(f_results, f_results.columns.to_list()[6:], 2)

In [ ]:
doubled_outliers

## Save outlier for M.

In [ ]:
filepath = '../open_work/internal_results' 
filename = os.path.join(filepath,'more_possible_outliers.csv') 
if not os.path.exists(filepath):
    # if filder doesn't exist, create it
    os.makedirs(filepath)
doubled_outliers.to_csv(filename)  

## Save off file of data

In [ ]:
filepath = '../open_work/internal_results' 
filename = os.path.join(filepath,'top_stitched_conformed.csv') 
if not os.path.exists(filepath):
    # if filder doesn't exist, create it
    os.makedirs(filepath)
f_results.to_csv(filename)  